In [7]:
import os
import zipfile
import dotenv
import subprocess
import pandas as pd
from typing import Dict, Any
import json
import yaml
import time
import glob
import hashlib
import shutil

config_train = "config/UrbanElementsReID_train.yml"  # "config/UAM_containers.yml"
#config_test = "config/UrbanElementsReID_test.yml"
config_test = "config/UrbanElementsReID_test_reduced.yml"
competition_name = "urban-reid-challenge"
submission_message = f"test trained submission"

################ Probably nothing has to be modified from now on ################
dataset_path = os.path.join('assets', 'datasets', competition_name)
with open(config_train, 'r') as f:
    hyperparams = yaml.load(f, Loader=yaml.BaseLoader)
model_path = os.path.join(hyperparams['LOG_ROOT'], hyperparams['LOG_NAME'])
experiment_id: int = int(time.time())

assert dotenv.load_dotenv('../../.env')
assert os.getenv('KAGGLE_USERNAME')

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [8]:
def calculate_params_hash(params: Dict[str, Any]) -> str:
    stringified = json.dumps({k: str(params[k]) for k in params}, sort_keys=True)
    return hashlib.md5(stringified.encode()).hexdigest()

In [ ]:
# Depednencies
# TODO: move to .def
!pip install torch torchvision torchaudio einops timm scikit-image opencv-python tensorboard yacs kaggle pyyaml

# Download the dataset
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path, exist_ok=True)
    api.competition_download_files(competition_name, path="./assets")
    with zipfile.ZipFile(f'./assets/{competition_name}.zip', 'r') as zip_ref:
        zip_ref.extractall(dataset_path)
    os.remove(f'./assets/{competition_name}.zip')
    print(f"Downloaded dataset for {competition_name}")
    
    %cd assets/datasets/urban-reid-challenge
    !mv ./image_query/image_query/* ./image_query/
    !rm -r ./image_query/image_query
    
    !mv ./image_test/image_test/* ./image_test/
    !rm -r ./image_test/image_test
    
    !mv ./image_train/image_train/* ./image_train/
    !rm -r ./image_train/image_train
    %cd ../../..
else:
    print(f"Dataset already existed")

# Generate reduced dataset
'''
cp -r urban-reid-challenge/ urban-reid-challenge-reduced/
cd urban-reid-challenge-reduced/
for file in image_query/0*.jpg; do
    num=$(basename "$file" .jpg)  # Extract number
    num=$((10#$num))  # Convert to decimal
    if ((num > 3)); then
        rm "$file"
    fi
done
for file in image_test/0*.jpg; do
    num=$(basename "$file" .jpg)  # Extract number
    num=$((10#$num))  # Convert to decimal
    if ((num > 10)); then
        rm "$file"
    fi
done

sed -i '4q' query.csv
sed -i '11q' test.csv

''';


# Download the model
os.makedirs('assets/models', exist_ok=True)
if not os.path.exists('assets/models/resnet50-19c8e357.pth'):
    !curl -o "assets/models/resnet50-19c8e357.pth" "https://download.pytorch.org/models/resnet50-19c8e357.pth"
else:
    print('model already existed')
if not os.path.exists('assets/models/jx_vit_base_p16_224-80ecf9dd.pth'):
    !curl -L -o 'assets/models/jx_vit_base_p16_224-80ecf9dd.pth'  'https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth'
else:
    print('model already existed')

# Train embedding model

In [136]:
!nvidia-smi

Fri Apr  4 02:20:49 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-16GB           On  | 00000000:18:00.0 Off |                    0 |
| N/A   38C    P0              25W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [113]:
assert os.path.exists('assets'), 'are you are in the right folder?'
assert os.getcwd().endswith('PAT'), 'are you are in the right folder?'
if os.path.exists(model_path):
    shutil.rmtree(model_path)
!python train.py --config_file {config_train}

2025-04-04 02:00:49,419 PAT INFO: Saving model in the path :assets/models/PAT
2025-04-04 02:00:49,419 PAT INFO: Namespace(config_file='config/UrbanElementsReID_train.yml', opts=[], local_rank=0)
2025-04-04 02:00:49,419 PAT INFO: Loaded configuration file config/UrbanElementsReID_train.yml
2025-04-04 02:00:49,419 PAT INFO: 
MODEL:
  PRETRAIN_CHOICE: 'imagenet'
  #PRETRAIN_PATH: "../../.cache/torch/hub/checkpoints" # root of pretrain path
  PRETRAIN_PATH: "assets/models" 
  IF_LABELSMOOTH: 'on'
  IF_WITH_CENTER: 'no'
  NAME: 'part_attention_vit'
  NO_MARGIN: True
  DEVICE_ID: ('0')
  TRANSFORMER_TYPE: 'vit_base_patch16_224_TransReID'
  STRIDE_SIZE: [16, 16]

INPUT:
  SIZE_TRAIN: [256,128]
  SIZE_TEST: [256,128]
  REA:
    ENABLED: False
  PIXEL_MEAN: [0.5, 0.5, 0.5]
  PIXEL_STD: [0.5, 0.5, 0.5]
  LGT: # Local Grayscale Transfomation
    DO_LGT: True
    PROB: 0.5

DATASETS:
  TRAIN: ('UrbanElementsReID',)
  TEST: ('UrbanElementsReID',)
  #ROOT_DIR: ('../../data') # root of datasets
  #RO

In [9]:
files = glob.glob(os.path.join(model_path, "part_attention_vit_*.pth"))
max_epoch = max([int(f.split('_')[-1].split('.')[0]) for f in files])
assert max_epoch > 0
assert os.path.exists(os.path.join(model_path, f'part_attention_vit_{max_epoch}.pth'))
print('max_epoch:', max_epoch)

max_epoch: 1


# Refine query images

for each image, generate refinements following naming like:
000001_refinement_A
000002_refinement_B
000002_refinement_C

save them in urban-reid-challenge-reduced/image_query/

Add those entires to urban-reid-challenge-reduced/image_query/query.csv

# Generate ranking

In [10]:
with open(config_test, 'r') as f:
    hyperparams_test = yaml.load(f, Loader=yaml.BaseLoader)
assert hyperparams_test['TEST']['WEIGHT'] == os.path.join(model_path, f'part_attention_vit_{max_epoch}.pth'), 'not testing with the trained model...'

In [26]:
!python update.py --config_file {config_test} --track {os.path.join(model_path, "track.txt")}

In [18]:
submission_file_name = os.path.join(model_path, "track_submission.csv")
assert os.path.exists(submission_file_name)

In [ ]:
!cp -r {model_path} {model_path}_backup_{experiment_id}

# Submission

In [133]:
assert os.path.exists(submission_file_name)
df = pd.read_csv(submission_file_name)
assert df.shape[1] == 2
assert df.shape[0] > 100

submission_message += f"; commit_hash: {subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode('ascii').strip()}"
submission_message += f"; hyperparameters_hash: {calculate_params_hash(hyperparams)}"
submission_message += f"; experiment_ID: {experiment_id}"
print(f'Submitting with message "{submission_message}"')

# Submit the file to the competition
# Uncomment only for actual submissions!
#api.competition_submit(submission_file_name, submission_message, competition_name)    